In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
os.chdir("src")

In [4]:
from anomaly_detection import *
from utils import *
from data_preparation import *
from constants import DATA_PATH, SEP_CSV
import matplotlib.pyplot as plt

In [5]:
os.chdir("../")

In [6]:
df = get_data()

### Step 1: Find registration all-time highs 

In [7]:
def find_all_time_high(df):
    max_value = df['c_VALUE'].max()
    all_time_high = df[df['c_VALUE'] == max_value]
    return all_time_high

In [8]:
all_time_high = find_all_time_high(df)
print("All-Time High:")
print(all_time_high)

All-Time High:
           c_FEATURE_1 c_FEATURE_2 c_FEATURE_3 c_FEATURE_4  c_VALUE
c_DATE                                                             
1990-01-01     Italien        Fiat       Hatch         Uno    50566


In [9]:
def find_all_time_high_keys(df, c_FEATURE_1, c_FEATURE_2, c_FEATURE_3, c_FEATURE_4):
    filter_condition = (
        (df['c_FEATURE_1'] == c_FEATURE_1) &
        (df['c_FEATURE_2'] == c_FEATURE_2) &
        (df['c_FEATURE_3'] == c_FEATURE_3) &
        (df['c_FEATURE_4'] == c_FEATURE_4)
    )
    filtered_data = df[filter_condition]
    max_value = filtered_data['c_VALUE'].max()
    max_timestamp = filtered_data.loc[filtered_data['c_VALUE'].idxmax()].name
    return max_value, max_timestamp

In [10]:
max_value, max_timestamp = find_all_time_high_keys(df, 'Belgien', 'Alfa Romeo', 'Hatch', '33')
print("Maximum value for the given keys:", max_value)
print("Timestamp of the maximum value:", max_timestamp)

Maximum value for the given keys: 396
Timestamp of the maximum value: 1992-01-01 00:00:00


### Step 2: Find lifecycle highs

In [11]:
def find_lifecycle_highs(df, c_FEATURE_1, lifecycle_threshold):
    filtered_df = df[df['c_FEATURE_1'] == c_FEATURE_1]  # Filter by c_FEATURE_1
    max_value_indices = filtered_df.groupby(['c_FEATURE_1', 'c_FEATURE_2', 'c_FEATURE_3', 'c_FEATURE_4'])['c_VALUE'].idxmax()
    max_rows = filtered_df.loc[max_value_indices]
    lifecycle_highs = max_rows[max_rows['c_VALUE'] > lifecycle_threshold]
    return lifecycle_highs

In [12]:
max_rows = find_lifecycle_highs(df, 'Deutschland', lifecycle_threshold=1000)
print("\nRows with Maximum Values:")
print(max_rows)


Rows with Maximum Values:
            c_FEATURE_1 c_FEATURE_2 c_FEATURE_3 c_FEATURE_4  c_VALUE
c_DATE                                                              
1987-11-01  Deutschland     Renault       Hatch          11     1737
1987-11-01  Deutschland     Peugeot       Hatch         205     2054
1987-11-01  Deutschland     Peugeot       Hatch         205     1254
1987-11-01  Deutschland       Mazda       Hatch         323     1041
1987-11-01  Deutschland         BMW   Limousine         3er     7507
...                 ...         ...         ...         ...      ...
2020-03-01  Deutschland    Mercedes         MPV    V-Klasse     1444
2020-03-01  Deutschland         BMW         SAV          X1     1365
2020-03-01  Deutschland         BMW         SAV          X3     1104
2020-03-01  Deutschland      Toyota       Hatch       Yaris     1365
2020-03-01  Deutschland     Hyundai       Hatch         i20     1214

[175261 rows x 5 columns]


### Step 3: Identify offer interruptions

In [13]:
def identify_offer_interruptions(df, c_FEATURE_1, interruption_threshold):
    filtered_df = df[df['c_FEATURE_1'] == c_FEATURE_1]  # Filter by c_FEATURE_1
    min_value_indices = filtered_df.groupby(['c_FEATURE_1', 'c_FEATURE_2', 'c_FEATURE_3', 'c_FEATURE_4'])['c_VALUE'].idxmin()
    min_rows = filtered_df.loc[min_value_indices]
    offer_interruptions = min_rows[min_rows['c_VALUE'] < interruption_threshold]
    return offer_interruptions

In [14]:
offer_interruptions = identify_offer_interruptions(df, 'Deutschland', interruption_threshold=5)
print("\nOffer Interruptions:")
print(offer_interruptions)


Offer Interruptions:
            c_FEATURE_1 c_FEATURE_2     c_FEATURE_3   c_FEATURE_4  c_VALUE
c_DATE                                                                    
1996-01-01  Deutschland     Peugeot           Hatch           106        0
1996-01-01  Deutschland     Peugeot           Hatch           106        0
1996-01-01  Deutschland       Mazda           Hatch           121        0
1996-01-01  Deutschland        Fiat       Limousine           126        1
1996-01-01  Deutschland  Alfa Romeo       Limousine           164        3
...                 ...         ...             ...           ...      ...
2019-01-01  Deutschland     Hyundai  Andere/Unspez.           i30        0
2019-01-01  Deutschland     Hyundai           Hatch           i30        1
2019-01-01  Deutschland     Hyundai           Hatch  i30 Fastback        4
2019-01-01  Deutschland     Hyundai             SAT          ix20        0
2019-01-01  Deutschland     Hyundai             SAV          ix55        1

[9

In [15]:
def identify_offer_interruptions_withAll(df, c_FEATURE_1, c_FEATURE_2, c_FEATURE_3, c_FEATURE_4, interruption_threshold):
    filtered_df = df[(df['c_FEATURE_1'] == c_FEATURE_1) &
                     (df['c_FEATURE_2'] == c_FEATURE_2) &
                     (df['c_FEATURE_3'] == c_FEATURE_3) &
                     (df['c_FEATURE_4'] == c_FEATURE_4)]  # Filter by c_FEATUREs
    offer_interruptions = filtered_df[filtered_df['c_VALUE'] < interruption_threshold]
    offer_interruptions = offer_interruptions.reset_index()
    return offer_interruptions

In [16]:
offer_interruptions = identify_offer_interruptions_withAll(df, 'Niederlande', 'PGO', 'Roadster', 'unspec.', interruption_threshold=3)
print("\nOffer Interruptions:")
print(offer_interruptions)


Offer Interruptions:
       c_DATE  c_FEATURE_1 c_FEATURE_2 c_FEATURE_3 c_FEATURE_4  c_VALUE
0  2004-01-01  Niederlande         PGO    Roadster     unspec.        0
1  2004-02-01  Niederlande         PGO    Roadster     unspec.        0
2  2004-03-01  Niederlande         PGO    Roadster     unspec.        0
3  2004-04-01  Niederlande         PGO    Roadster     unspec.        0
4  2004-05-01  Niederlande         PGO    Roadster     unspec.        0
5  2004-06-01  Niederlande         PGO    Roadster     unspec.        0
6  2005-06-01  Niederlande         PGO    Roadster     unspec.        2
7  2004-07-01  Niederlande         PGO    Roadster     unspec.        0
8  2005-07-01  Niederlande         PGO    Roadster     unspec.        0
9  2005-08-01  Niederlande         PGO    Roadster     unspec.        1
10 2005-09-01  Niederlande         PGO    Roadster     unspec.        1
11 2004-10-01  Niederlande         PGO    Roadster     unspec.        1
12 2005-10-01  Niederlande         PGO    

### Step 4: Develop a dynamic method

In [17]:
def identify_events(df, c_FEATURE_1, lifecycle_threshold, interruption_threshold):
    all_time_high = find_all_time_high(df)
    lifecycle_highs = find_lifecycle_highs(df, c_FEATURE_1, lifecycle_threshold)
    offer_interruptions = identify_offer_interruptions(df, c_FEATURE_1, interruption_threshold)
    
    events = pd.concat([all_time_high, lifecycle_highs, offer_interruptions])
    events = events.drop_duplicates()
    
    return events

In [18]:
# Example 4: Identify events
events = identify_events(df, c_FEATURE_1 = 'Deutschland', lifecycle_threshold=1000, interruption_threshold=0.1)
print("\nEvents:")
print(events)


Events:
            c_FEATURE_1 c_FEATURE_2     c_FEATURE_3 c_FEATURE_4  c_VALUE
c_DATE                                                                  
1990-01-01      Italien        Fiat           Hatch         Uno    50566
1987-11-01  Deutschland     Renault           Hatch          11     1737
1987-11-01  Deutschland     Peugeot           Hatch         205     2054
1987-11-01  Deutschland     Peugeot           Hatch         205     1254
1987-11-01  Deutschland       Mazda           Hatch         323     1041
...                 ...         ...             ...         ...      ...
2013-07-01  Deutschland      Toyota           Hatch          IQ        0
2001-02-01  Deutschland          VW  Andere/Unspez.         Gol        0
1983-05-01  Deutschland    Vauxhall           Kombi    Chevette        0
2015-02-01  Deutschland       Volvo           Kombi        XC70        0
2022-08-01  Deutschland      Zhidou           Hatch     ZD1/ZD2        0

[32591 rows x 5 columns]
